# Convert JSON Voting Data to SQLite Database

This notebook reads all Swiss voting JSON files and loads them into a well-structured SQLite database for efficient querying and analysis.

In [ ]:
import json
import sqlite3
from pathlib import Path
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import hashlib

## Database Schema Design

The database is designed with the following tables:

1. **votings** - Master table for all voting events
2. **proposals** (vorlagen) - Individual referendum proposals within a voting
3. **cantons** - Canton reference data
4. **districts** - District reference data  
5. **municipalities** - Municipality reference data with validity dates
6. **voting_results** - Actual voting results at different geographic levels
7. **spatial_references** - Track geographic structure changes over time

In [ ]:
# Database configuration
DB_PATH = Path('data/swiss_votings.db')
VOTES_DIR = Path('data/votes')

# Remove existing database if it exists
if DB_PATH.exists():
    DB_PATH.unlink()
    print(f"Removed existing database: {DB_PATH}")

print(f"Database will be created at: {DB_PATH}")
print(f"Reading JSON files from: {VOTES_DIR}")

In [ ]:
# Create database connection
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Enable foreign keys
cursor.execute("PRAGMA foreign_keys = ON")

print("Database connection established")

## Create Database Tables

In [ ]:
# Drop existing tables if they exist (for clean start)
tables_to_drop = [
    'voting_results', 'spatial_references', 'proposals', 'votings',
    'municipalities', 'districts', 'cantons'
]

for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")

print("Cleared existing tables")

In [ ]:
# Create tables with proper schema

# 1. Votings table - master list of all voting events
cursor.execute("""
CREATE TABLE votings (
    voting_id INTEGER PRIMARY KEY AUTOINCREMENT,
    voting_date TEXT NOT NULL UNIQUE,  -- Format: YYYYMMDD
    timestamp TEXT,
    source_file TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# 2. Proposals table - individual referendum questions
cursor.execute("""
CREATE TABLE proposals (
    proposal_id INTEGER PRIMARY KEY AUTOINCREMENT,
    voting_id INTEGER NOT NULL,
    vorlage_id INTEGER,
    title_de TEXT,
    title_fr TEXT,
    title_it TEXT,
    title_rm TEXT,
    title_en TEXT,
    proposal_type INTEGER,
    angenommen BOOLEAN,
    doppeltes_mehr BOOLEAN,
    FOREIGN KEY (voting_id) REFERENCES votings(voting_id)
)
""")

# 3. Cantons table
cursor.execute("""
CREATE TABLE cantons (
    canton_id TEXT PRIMARY KEY,  -- Using geoLevelnummer as ID
    canton_name TEXT NOT NULL,
    first_seen_date TEXT,
    last_seen_date TEXT
)
""")

# 4. Districts table
cursor.execute("""
CREATE TABLE districts (
    district_id TEXT PRIMARY KEY,  -- Using geoLevelnummer as ID
    district_name TEXT NOT NULL,
    canton_id TEXT,
    first_seen_date TEXT,
    last_seen_date TEXT,
    FOREIGN KEY (canton_id) REFERENCES cantons(canton_id)
)
""")

# 5. Municipalities table with temporal tracking
cursor.execute("""
CREATE TABLE municipalities (
    municipality_id TEXT PRIMARY KEY,  -- Using geoLevelnummer as ID
    municipality_name TEXT NOT NULL,
    district_id TEXT,
    canton_id TEXT,
    parent_id TEXT,  -- Parent district/canton reference
    first_seen_date TEXT,  -- First voting date this municipality appears
    last_seen_date TEXT,   -- Last voting date this municipality appears
    successor_id TEXT,     -- ID of municipality this merged into (if applicable)
    FOREIGN KEY (district_id) REFERENCES districts(district_id),
    FOREIGN KEY (canton_id) REFERENCES cantons(canton_id)
)
""")

# 6. Voting results table - stores actual voting data
cursor.execute("""
CREATE TABLE voting_results (
    result_id INTEGER PRIMARY KEY AUTOINCREMENT,
    voting_id INTEGER NOT NULL,
    proposal_id INTEGER NOT NULL,
    geo_level TEXT NOT NULL,  -- 'switzerland', 'canton', 'district', 'municipality'
    geo_id TEXT NOT NULL,      -- ID of the geographic entity
    geo_name TEXT,
    
    -- Voting statistics
    ja_stimmen_absolut INTEGER,
    nein_stimmen_absolut INTEGER,
    ja_stimmen_prozent REAL,
    stimmbeteiligung_prozent REAL,
    gueltige_stimmen INTEGER,
    eingelegte_stimmzettel INTEGER,
    anzahl_stimmberechtigte INTEGER,
    gebiet_ausgezaehlt BOOLEAN,
    
    FOREIGN KEY (voting_id) REFERENCES votings(voting_id),
    FOREIGN KEY (proposal_id) REFERENCES proposals(proposal_id)
)
""")

# 7. Spatial references table - track geographic structure changes
cursor.execute("""
CREATE TABLE spatial_references (
    reference_id INTEGER PRIMARY KEY AUTOINCREMENT,
    voting_id INTEGER NOT NULL,
    spatial_unit TEXT NOT NULL,  -- 'kant', 'bezk', 'voge'
    spatial_date TEXT NOT NULL,  -- Date this spatial structure was valid
    FOREIGN KEY (voting_id) REFERENCES votings(voting_id)
)
""")

conn.commit()
print("All tables created successfully")

## Create Indexes for Performance

In [ ]:
# Create indexes for common query patterns
indexes = [
    "CREATE INDEX idx_votings_date ON votings(voting_date)",
    "CREATE INDEX idx_proposals_voting ON proposals(voting_id)",
    "CREATE INDEX idx_results_voting ON voting_results(voting_id)",
    "CREATE INDEX idx_results_proposal ON voting_results(proposal_id)",
    "CREATE INDEX idx_results_geo ON voting_results(geo_level, geo_id)",
    "CREATE INDEX idx_municipalities_dates ON municipalities(first_seen_date, last_seen_date)",
    "CREATE INDEX idx_municipalities_parent ON municipalities(parent_id)",
    "CREATE INDEX idx_spatial_voting ON spatial_references(voting_id)"
]

for idx_sql in indexes:
    cursor.execute(idx_sql)

conn.commit()
print(f"Created {len(indexes)} indexes for query optimization")

## Load JSON Data into Database

In [ ]:
# Get all JSON files
json_files = sorted(VOTES_DIR.glob('sd-t-17-02-*-eidgAbstimmung.json'))
print(f"Found {len(json_files)} JSON files to process")

# Display date range
if json_files:
    first_date = json_files[0].name.split('-')[5].replace('eidgAbstimmung.json', '')
    last_date = json_files[-1].name.split('-')[5].replace('eidgAbstimmung.json', '')
    print(f"Date range: {first_date} to {last_date}")

In [ ]:
def process_voting_file(file_path, conn):
    """Process a single voting JSON file and insert into database"""
    
    cursor = conn.cursor()
    
    try:
        # Load JSON data
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        voting_date = data.get('abstimmtag', '')
        timestamp = data.get('timestamp', '')
        
        # 1. Insert voting record
        cursor.execute("""
            INSERT OR IGNORE INTO votings (voting_date, timestamp, source_file)
            VALUES (?, ?, ?)
        """, (voting_date, timestamp, file_path.name))
        
        voting_id = cursor.lastrowid
        if voting_id == 0:  # If already exists, get the ID
            cursor.execute("SELECT voting_id FROM votings WHERE voting_date = ?", (voting_date,))
            voting_id = cursor.fetchone()[0]
        
        # 2. Insert spatial references
        if 'spatial_reference' in data:
            for ref in data['spatial_reference']:
                cursor.execute("""
                    INSERT INTO spatial_references (voting_id, spatial_unit, spatial_date)
                    VALUES (?, ?, ?)
                """, (voting_id, ref.get('spatial_unit'), ref.get('spatial_date')))
        
        # 3. Process proposals (vorlagen)
        if 'schweiz' in data and 'vorlagen' in data['schweiz']:
            for vorlage in data['schweiz']['vorlagen']:
                # Extract titles
                titles = {}
                if 'vorlagenTitel' in vorlage:
                    for title in vorlage['vorlagenTitel']:
                        lang_key = title.get('langKey', '')
                        titles[f'title_{lang_key}'] = title.get('text', '')
                
                # Insert proposal
                cursor.execute("""
                    INSERT INTO proposals (
                        voting_id, vorlage_id, title_de, title_fr, title_it, title_rm, title_en,
                        proposal_type, angenommen, doppeltes_mehr
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    voting_id,
                    vorlage.get('vorlagenId'),
                    titles.get('title_de'),
                    titles.get('title_fr'),
                    titles.get('title_it'),
                    titles.get('title_rm'),
                    titles.get('title_en'),
                    vorlage.get('vorlagenArtId'),
                    vorlage.get('vorlageAngenommen'),
                    vorlage.get('doppeltesMehr')
                ))
                
                proposal_id = cursor.lastrowid
                
                # 4. Insert Switzerland-level results
                if 'resultat' in vorlage:
                    res = vorlage['resultat']
                    cursor.execute("""
                        INSERT INTO voting_results (
                            voting_id, proposal_id, geo_level, geo_id, geo_name,
                            ja_stimmen_absolut, nein_stimmen_absolut, ja_stimmen_prozent,
                            stimmbeteiligung_prozent, gueltige_stimmen, eingelegte_stimmzettel,
                            anzahl_stimmberechtigte, gebiet_ausgezaehlt
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """, (
                        voting_id, proposal_id, 'switzerland', '0', 'Schweiz',
                        res.get('jaStimmenAbsolut'), res.get('neinStimmenAbsolut'),
                        res.get('jaStimmenInProzent'), res.get('stimmbeteiligungInProzent'),
                        res.get('gueltigeStimmen'), res.get('eingelegteStimmzettel'),
                        res.get('anzahlStimmberechtigte'), res.get('gebietAusgezaehlt')
                    ))
                
                # 5. Process cantons
                if 'kantone' in vorlage:
                    for kanton in vorlage['kantone']:
                        canton_id = kanton.get('geoLevelnummer')
                        canton_name = kanton.get('geoLevelname')
                        
                        # Update canton reference
                        cursor.execute("""
                            INSERT OR IGNORE INTO cantons (canton_id, canton_name)
                            VALUES (?, ?)
                        """, (canton_id, canton_name))
                        
                        # Update seen dates
                        cursor.execute("""
                            UPDATE cantons 
                            SET first_seen_date = MIN(IFNULL(first_seen_date, ?), ?),
                                last_seen_date = MAX(IFNULL(last_seen_date, ?), ?)
                            WHERE canton_id = ?
                        """, (voting_date, voting_date, voting_date, voting_date, canton_id))
                        
                        # Insert canton results
                        if 'resultat' in kanton:
                            res = kanton['resultat']
                            cursor.execute("""
                                INSERT INTO voting_results (
                                    voting_id, proposal_id, geo_level, geo_id, geo_name,
                                    ja_stimmen_absolut, nein_stimmen_absolut, ja_stimmen_prozent,
                                    stimmbeteiligung_prozent, gueltige_stimmen, eingelegte_stimmzettel,
                                    anzahl_stimmberechtigte, gebiet_ausgezaehlt
                                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            """, (
                                voting_id, proposal_id, 'canton', canton_id, canton_name,
                                res.get('jaStimmenAbsolut'), res.get('neinStimmenAbsolut'),
                                res.get('jaStimmenInProzent'), res.get('stimmbeteiligungInProzent'),
                                res.get('gueltigeStimmen'), res.get('eingelegteStimmzettel'),
                                res.get('anzahlStimmberechtigte'), res.get('gebietAusgezaehlt')
                            ))
                        
                        # 6. Process districts within canton
                        if 'bezirke' in kanton:
                            for bezirk in kanton['bezirke']:
                                district_id = bezirk.get('geoLevelnummer')
                                district_name = bezirk.get('geoLevelname')
                                
                                # Update district reference
                                cursor.execute("""
                                    INSERT OR IGNORE INTO districts (district_id, district_name, canton_id)
                                    VALUES (?, ?, ?)
                                """, (district_id, district_name, canton_id))
                                
                                # Update seen dates
                                cursor.execute("""
                                    UPDATE districts 
                                    SET first_seen_date = MIN(IFNULL(first_seen_date, ?), ?),
                                        last_seen_date = MAX(IFNULL(last_seen_date, ?), ?)
                                    WHERE district_id = ?
                                """, (voting_date, voting_date, voting_date, voting_date, district_id))
                                
                                # Insert district results
                                if 'resultat' in bezirk:
                                    res = bezirk['resultat']
                                    cursor.execute("""
                                        INSERT INTO voting_results (
                                            voting_id, proposal_id, geo_level, geo_id, geo_name,
                                            ja_stimmen_absolut, nein_stimmen_absolut, ja_stimmen_prozent,
                                            stimmbeteiligung_prozent, gueltige_stimmen, eingelegte_stimmzettel,
                                            anzahl_stimmberechtigte, gebiet_ausgezaehlt
                                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                                    """, (
                                        voting_id, proposal_id, 'district', district_id, district_name,
                                        res.get('jaStimmenAbsolut'), res.get('neinStimmenAbsolut'),
                                        res.get('jaStimmenInProzent'), res.get('stimmbeteiligungInProzent'),
                                        res.get('gueltigeStimmen'), res.get('eingelegteStimmzettel'),
                                        res.get('anzahlStimmberechtigte'), res.get('gebietAusgezaehlt')
                                    ))
                
                # 7. Process municipalities
                if 'gemeinden' in vorlage:
                    for gemeinde in vorlage['gemeinden']:
                        municipality_id = gemeinde.get('geoLevelnummer')
                        municipality_name = gemeinde.get('geoLevelname')
                        parent_id = gemeinde.get('geoLevelParentnummer')
                        
                        # Update municipality reference
                        cursor.execute("""
                            INSERT OR IGNORE INTO municipalities (
                                municipality_id, municipality_name, parent_id
                            ) VALUES (?, ?, ?)
                        """, (municipality_id, municipality_name, parent_id))
                        
                        # Update seen dates
                        cursor.execute("""
                            UPDATE municipalities 
                            SET first_seen_date = MIN(IFNULL(first_seen_date, ?), ?),
                                last_seen_date = MAX(IFNULL(last_seen_date, ?), ?)
                            WHERE municipality_id = ?
                        """, (voting_date, voting_date, voting_date, voting_date, municipality_id))
                        
                        # Insert municipality results
                        if 'resultat' in gemeinde:
                            res = gemeinde['resultat']
                            cursor.execute("""
                                INSERT INTO voting_results (
                                    voting_id, proposal_id, geo_level, geo_id, geo_name,
                                    ja_stimmen_absolut, nein_stimmen_absolut, ja_stimmen_prozent,
                                    stimmbeteiligung_prozent, gueltige_stimmen, eingelegte_stimmzettel,
                                    anzahl_stimmberechtigte, gebiet_ausgezaehlt
                                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            """, (
                                voting_id, proposal_id, 'municipality', municipality_id, municipality_name,
                                res.get('jaStimmenAbsolut'), res.get('neinStimmenAbsolut'),
                                res.get('jaStimmenInProzent'), res.get('stimmbeteiligungInProzent'),
                                res.get('gueltigeStimmen'), res.get('eingelegteStimmzettel'),
                                res.get('anzahlStimmberechtigte'), res.get('gebietAusgezaehlt')
                            ))
        
        conn.commit()
        return True, None
        
    except Exception as e:
        conn.rollback()
        return False, str(e)

print("Processing function defined")

In [ ]:
# Process all JSON files
success_count = 0
error_count = 0
errors = []

print(f"\nProcessing {len(json_files)} files...\n")

for file_path in tqdm(json_files, desc="Loading JSON files"):
    success, error = process_voting_file(file_path, conn)
    
    if success:
        success_count += 1
    else:
        error_count += 1
        errors.append(f"{file_path.name}: {error}")

print(f"\n✓ Successfully processed: {success_count} files")
print(f"✗ Errors encountered: {error_count} files")

if errors:
    print("\nErrors:")
    for error in errors[:5]:  # Show first 5 errors
        print(f"  - {error}")

## Verify Database Content

In [ ]:
# Check record counts
tables = ['votings', 'proposals', 'cantons', 'districts', 'municipalities', 'voting_results', 'spatial_references']

print("Database Statistics:")
print("=" * 40)

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"{table:20} {count:10,} records")

print("=" * 40)

In [ ]:
# Sample queries to verify data
print("\nSample Data Verification:")
print("=" * 60)

# 1. Recent votings
print("\n1. Recent votings:")
cursor.execute("""
    SELECT voting_date, COUNT(DISTINCT p.proposal_id) as num_proposals
    FROM votings v
    LEFT JOIN proposals p ON v.voting_id = p.voting_id
    GROUP BY v.voting_date
    ORDER BY v.voting_date DESC
    LIMIT 5
""")

for row in cursor.fetchall():
    print(f"  {row[0]}: {row[1]} proposals")

# 2. Municipality changes over time
print("\n2. Municipality lifecycle:")
cursor.execute("""
    SELECT 
        COUNT(CASE WHEN first_seen_date = '20000312' THEN 1 END) as existed_2000,
        COUNT(CASE WHEN last_seen_date >= '20250209' THEN 1 END) as exists_2025,
        COUNT(CASE WHEN first_seen_date > '20000312' THEN 1 END) as created_after_2000,
        COUNT(CASE WHEN last_seen_date < '20250209' THEN 1 END) as disappeared
    FROM municipalities
""")

stats = cursor.fetchone()
print(f"  Existed in 2000: {stats[0]}")
print(f"  Exists in 2025: {stats[1]}")
print(f"  Created after 2000: {stats[2]}")
print(f"  Disappeared: {stats[3]}")

# 3. Sample proposal
print("\n3. Sample proposal:")
cursor.execute("""
    SELECT v.voting_date, p.title_de
    FROM proposals p
    JOIN votings v ON p.voting_id = v.voting_id
    WHERE p.title_de IS NOT NULL
    ORDER BY v.voting_date DESC
    LIMIT 1
""")

row = cursor.fetchone()
if row:
    print(f"  {row[0]}: {row[1][:80]}...")

## Create Views for Common Queries

In [ ]:
# Create useful views for analysis

# 1. Municipality history view
cursor.execute("""
CREATE VIEW IF NOT EXISTS municipality_history AS
SELECT 
    m.municipality_id,
    m.municipality_name,
    m.first_seen_date,
    m.last_seen_date,
    m.parent_id,
    d.district_name,
    c.canton_name,
    CASE 
        WHEN m.last_seen_date < '20250209' THEN 'disappeared'
        WHEN m.first_seen_date > '20000312' THEN 'new'
        ELSE 'continuous'
    END as status
FROM municipalities m
LEFT JOIN districts d ON m.parent_id = d.district_id
LEFT JOIN cantons c ON d.canton_id = c.canton_id
""")

# 2. Voting summary view
cursor.execute("""
CREATE VIEW IF NOT EXISTS voting_summary AS
SELECT 
    v.voting_date,
    p.proposal_id,
    p.title_de,
    p.angenommen,
    r.ja_stimmen_prozent as national_ja_prozent,
    r.stimmbeteiligung_prozent as national_beteiligung
FROM votings v
JOIN proposals p ON v.voting_id = p.voting_id
JOIN voting_results r ON p.proposal_id = r.proposal_id AND r.geo_level = 'switzerland'
ORDER BY v.voting_date DESC, p.proposal_id
""")

conn.commit()
print("Views created successfully")

## Export Database Information

In [ ]:
# Generate database documentation
doc_path = Path('data/database_documentation.md')

documentation = f"""
# Swiss Voting Database Documentation

Generated: {datetime.now().isoformat()}

## Database Location
- Path: {DB_PATH}
- Size: {DB_PATH.stat().st_size / (1024*1024):.2f} MB

## Tables

### votings
- Master table for all voting events
- Primary Key: voting_id
- Unique: voting_date (YYYYMMDD format)

### proposals
- Individual referendum questions within each voting
- Links to: votings (voting_id)
- Contains multilingual titles (de, fr, it, rm, en)

### municipalities
- All municipalities with temporal tracking
- Tracks first_seen_date and last_seen_date
- Can identify mergers via successor_id field

### voting_results
- Actual voting data at all geographic levels
- geo_level: 'switzerland', 'canton', 'district', 'municipality'
- Contains absolute numbers and percentages

## Sample Queries

```sql
-- Find disappeared municipalities
SELECT municipality_name, first_seen_date, last_seen_date
FROM municipalities
WHERE last_seen_date < '20250209'
ORDER BY last_seen_date DESC;

-- Get voting results for a specific municipality over time
SELECT v.voting_date, p.title_de, r.ja_stimmen_prozent
FROM voting_results r
JOIN votings v ON r.voting_id = v.voting_id
JOIN proposals p ON r.proposal_id = p.proposal_id
WHERE r.geo_level = 'municipality' 
  AND r.geo_name = 'Zürich'
ORDER BY v.voting_date;
```

## Notes
- Municipality IDs and names may change over time due to mergers
- Use first_seen_date and last_seen_date to track municipality lifecycle
- spatial_references table tracks when geographic structures changed
"""

with open(doc_path, 'w') as f:
    f.write(documentation.strip())

print(f"Documentation saved to: {doc_path}")
print(f"Database ready for analysis at: {DB_PATH}")

In [ ]:
# Close database connection
conn.close()
print("\nDatabase connection closed.")
print(f"\n✅ Database successfully created at: {DB_PATH}")
print("\nYou can now:")
print("1. Query the database using SQL")
print("2. Join voting results with municipality changes from the Excel file")
print("3. Track municipality mergers using first_seen_date and last_seen_date")
print("4. Aggregate historical data to current municipality structures")